#### STEPS

- **Model Initialisation:** Instantiate the Large Language Model (LLM) object
---
- **Schema Definition:** Construct the Pydantic class to define the required data structure
---
- **Parser Creation:** Instantiate the PydanticOutputParser & bind it to the Pydantic class
---
- **Prompt Construction:** Define the ChatPromptTemplate & inject the parser's instructions

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Read
df = pd.read_csv('Data/cleaned_reviews.csv')

# Total Reviews
print(f"> Total Reviews: {df.shape[0]:,}")

# SLICE (100 Reviews: en)
df = df.head(100)

# View
display(df.head())

> Total Reviews: 1,054


,reviews_text
0,the hotel was great. staff went above and beyond bringing hot tea to room at 11:30 pm at no charge!
1,"a wonderful hotel - would definitely stay there whenever we are in alexandria again. wonderful staff. we remarked that the first room was very far from the elevator, and they immediately moved us to a much nicer/larger room across from the elevator without question. i hope i am able to stay at a kimpton hotel on future trips."
2,"i needed a place to stay in va that was close to or in alexandria and the monaco was available for a great value. i enjoyed my stay, room was comfortable and clean. was disappointed on parking cost but i was able to work around that. very convenient to old town."
3,"the hotel itself was very nice. the desk staff were very attentive and helpful. the rooms were nice but not dusted/cleaned adequately each day. the problems we had were few but definitely impacted whether we would stay there again. the hallways and elevators were not air-conditioned and were in the high 80's in july. also, with no complimentary breakfast offered, the free coffee was in high demand. the coffee hours ended at 9:00 a.m. and the coffee was always empty, missing ""to go"" cups, or out of cream and sugar. a daily conversation with the desk staff on this issue grew tiresome with our 7 day stay. look elsewhere if service is important to you."
4,"we arrive around 1 am, for a 5 night stay, and the lady could not find our reservation. after a while she she found it but could not tell us a thing about our expedia gold benefits. she told us it was not her problem a we should ask the manager eventually."


#### MODEL

In [3]:
from langchain.chat_models import init_chat_model

# MODEL
model = init_chat_model(model="claude-haiku-4-5-20251001", temperature=0)

#### PYDANTIC MODEL

In [4]:
# Import BaseModel to construct the primary data structure & Field to attach LLM instructions
from pydantic import BaseModel, Field

# Import Literal to enforce strict string matching boundaries
from typing import Literal

# Define the extraction class, inheriting from BaseModel
class ReviewSentiment(BaseModel):
    
    # Create 'sentiment' variable & restrict permitted outputs to the three specific labels
    sentiment: Literal["Positive", "Negative", "Neutral"] = Field(
        
        # Describe operational directive the LLM will follow
        description="The overall sentiment of the hotel review. You must choose exactly one of the three options."
    )

#### PYDANTIC PARSER

In [5]:
from langchain_core.output_parsers import PydanticOutputParser

# Initialise OUTPUT parser with defined schema
parser = PydanticOutputParser(pydantic_object=ReviewSentiment)

#### PROMPT

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# Define the explicit instructions for the LLM
prompt = ChatPromptTemplate.from_template(
    "Analyse the following hotel review. \n"
    
    # The placeholder below will hold the rigid pydantic rules
    "{format_instructions}\n"

    # The placeholder below will hold the row data from the DataFrame
    "Review: {review}"
)

# INJECT PYDANTIC PARSER RULES INTO PROMPT
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

#### CREATE THE CHAIN

In [7]:
# Prompt -> LLM -> Parser
chain = prompt | model | parser

#### EXECUTION FUNCTION

In [8]:
# Execution function
def process_review(text):
    
    # > EXECUTE THE CHAIN
    
    try:
    
        # The chain outputs the Pydantic ReviewInsight object
        result = chain.invoke({"review": text})
    
        # Return the specific variables as a Pandas Series
        return pd.Series([result.sentiment.upper()])
    
    except Exception:
        return pd.Series(["ERROR"])

In [9]:
# Apply the execution function to the review column
df['sentiment'] = df['reviews_text'].apply(process_review)

In [10]:
# SENTIMENT COUNT
df['sentiment'].value_counts()

sentiment
POSITIVE    83
NEGATIVE     9
NEUTRAL      8
Name: count, dtype: int64

In [12]:
# View
display(df.head(15))

,reviews_text,sentiment
0,the hotel was great. staff went above and beyond bringing hot tea to room at 11:30 pm at no charge!,POSITIVE
1,"a wonderful hotel - would definitely stay there whenever we are in alexandria again. wonderful staff. we remarked that the first room was very far from the elevator, and they immediately moved us to a much nicer/larger room across from the elevator without question. i hope i am able to stay at a kimpton hotel on future trips.",POSITIVE
2,"i needed a place to stay in va that was close to or in alexandria and the monaco was available for a great value. i enjoyed my stay, room was comfortable and clean. was disappointed on parking cost but i was able to work around that. very convenient to old town.",POSITIVE
3,"the hotel itself was very nice. the desk staff were very attentive and helpful. the rooms were nice but not dusted/cleaned adequately each day. the problems we had were few but definitely impacted whether we would stay there again. the hallways and elevators were not air-conditioned and were in the high 80's in july. also, with no complimentary breakfast offered, the free coffee was in high demand. the coffee hours ended at 9:00 a.m. and the coffee was always empty, missing ""to go"" cups, or out of cream and sugar. a daily conversation with the desk staff on this issue grew tiresome with our 7 day stay. look elsewhere if service is important to you.",NEGATIVE
4,"we arrive around 1 am, for a 5 night stay, and the lady could not find our reservation. after a while she she found it but could not tell us a thing about our expedia gold benefits. she told us it was not her problem a we should ask the manager eventually.",NEGATIVE
5,"excellent location, good value, clean and comfortable room. what more could you ask for?!",POSITIVE
6,"overall the stay was fine. room was clean, bed & pillow were comfortable, nice toiletries and towels. check in was a little slow, but staff in the lobby were nice. however, the ac unit is loud & in the room. the mini fridge is somewhat noisy. and the toilet kept running every 10 minutes (loudest filling toilet ever). and the headboard had seen some damage (fake leather). the location is great though. short walk to harbor area, lots of restaurants, etc.",NEUTRAL
7,"the room was very nice and clean. the hotel was uniquely decorated and stylish. the location was nice if you plan to go out in old town, but a good 20+ minute walk to public transportation and a hike to get into dc.",POSITIVE
8,"the hotel exceeded our expectations. our room and the hotel was well kept, clean and the decor tasteful.the 5 p.m. manager hosted wine tasting is an excellent way to return to the hotel after a nice day out to see the sites and touring. it was also a great ice breaker to meet other guests (or not).to top it off this particular property is perfectly located on king street in historic alexandria surrounded by shops and excellent choices of restaurants.we will definitely stay here again and will recommend to friends.",POSITIVE
9,"from check in to check out the staff goes out of its way to insure that your stay is problem free. beautiful rooms, excellent staff, great location. we go to alexandria 2-3 times a year to enjoy the history and would not think of staying elsewhere. it is located right in the heart of old alexandria and within walking distance of everything.",POSITIVE
